In [6]:
# clean
import pandas as pd
import numpy as np
from pathlib import Path
import gc, warnings
warnings.filterwarnings('ignore')

INPUT_DIR = Path('/kaggle/input/ids-intrusion-csv')      # folder chứa 10 CSV gốc
CLEANED_DIR = Path('/kaggle/working/cleaned_data')       # output
CLEANED_DIR.mkdir(exist_ok=True, parents=True)

CHUNK_SIZE = 500_000

print("="*70)
print("CLEAN CICIDS2018 → BINARY (Benign=0, Attack=1)")
print("="*70)

cols_to_remove = [
    'Src IP', 'Src_IP', 'Source IP', 
    'Dst IP', 'Dst_IP', 'Destination IP',
    'Flow ID', 'Flow_ID',
    'Timestamp',
    'Src Port', 'Src_Port',
    'Dst Port', 'Dst_Port'
]

def standardize_columns(df):
    df.columns = df.columns.str.strip()
    rename_map = {
        'Src IP': 'Src_IP',
        'Dst IP': 'Dst_IP',
        'Src Port': 'Src_Port',
        'Dst Port': 'Dst_Port',
        'Flow ID': 'Flow_ID',
        ' Label': 'Label',
        'Label ': 'Label',
    }
    df.rename(columns=rename_map, inplace=True)
    df.columns = df.columns.str.replace(' ', '_')
    return df

def clean_chunk(chunk):
    chunk = standardize_columns(chunk)
    if 'Label' not in chunk.columns:
        return None
    
    # drop dup
    chunk = chunk.drop_duplicates()

    # bỏ dòng quá nhiều NaN
    missing_counts = chunk.isnull().sum(axis=1)
    chunk = chunk[missing_counts <= len(chunk.columns) * 0.3]

    # fill NaN + inf cho numeric
    num_cols = chunk.select_dtypes(include=[np.number]).columns
    chunk.replace([np.inf, -np.inf], np.nan, inplace=True)
    for col in num_cols:
        if chunk[col].isnull().any():
            chunk[col].fillna(chunk[col].median(), inplace=True)

    # binary label
    chunk['Label'] = chunk['Label'].str.strip()
    chunk['Label_Binary'] = chunk['Label'].apply(lambda x: 0 if x == 'Benign' else 1)

    # bỏ cột IP/port/timestamp
    drop_cols = [c for c in cols_to_remove if c in chunk.columns]
    if drop_cols:
        chunk = chunk.drop(columns=drop_cols)

    # thay Label gốc bằng Label_Binary
    chunk = chunk.drop(columns=['Label'])
    chunk.rename(columns={'Label_Binary': 'Label'}, inplace=True)

    return chunk

# chạy qua từng file
csv_files = sorted(INPUT_DIR.glob('*.csv'))
print(f"Found {len(csv_files)} raw files\n")

total_benign, total_attack = 0, 0

for i, fp in enumerate(csv_files, 1):
    print(f"[{i:>2}/{len(csv_files)}] {fp.name}")
    cleaned_chunks = []
    b_cnt, a_cnt = 0, 0

    for chunk in pd.read_csv(fp, chunksize=CHUNK_SIZE, low_memory=False):
        c = clean_chunk(chunk)
        if c is not None and len(c) > 0:
            b_cnt += (c['Label'] == 0).sum()
            a_cnt += (c['Label'] == 1).sum()
            cleaned_chunks.append(c)
        del chunk; gc.collect()

    if not cleaned_chunks:
        print("  → no data after cleaning, skip.")
        continue

    df_final = pd.concat(cleaned_chunks, ignore_index=True)
    out_path = CLEANED_DIR / f"cleaned_{fp.stem}.csv"
    df_final.to_csv(out_path, index=False)

    print(f"  → saved {len(df_final):,} rows  (Benign: {b_cnt:,}, Attack: {a_cnt:,})\n")
    total_benign += b_cnt
    total_attack += a_cnt

    del cleaned_chunks, df_final; gc.collect()

print("="*70)
print("CLEAN DONE")
print(f"Total Benign: {total_benign:,}")
print(f"Total Attack: {total_attack:,}")
print(f"Ratio: {total_benign/(total_benign+total_attack)*100:.1f}% Benign")
print(f"Output dir: {CLEANED_DIR}")
print("="*70)


CLEAN CICIDS2018 → BINARY (Benign=0, Attack=1)
Found 10 raw files

[ 1/10] 02-14-2018.csv
  → saved 823,447 rows  (Benign: 666,773, Attack: 156,674)

[ 2/10] 02-15-2018.csv
  → saved 1,046,810 rows  (Benign: 995,070, Attack: 51,740)

[ 3/10] 02-16-2018.csv
  → saved 911,350 rows  (Benign: 446,772, Attack: 464,578)

[ 4/10] 02-20-2018.csv
  → saved 7,948,746 rows  (Benign: 7,372,555, Attack: 576,191)

[ 5/10] 02-21-2018.csv
  → saved 1,031,019 rows  (Benign: 360,827, Attack: 670,192)

[ 6/10] 02-22-2018.csv
  → saved 1,046,432 rows  (Benign: 1,046,070, Attack: 362)

[ 7/10] 02-23-2018.csv
  → saved 1,046,620 rows  (Benign: 1,046,054, Attack: 566)

[ 8/10] 02-28-2018.csv
  → saved 609,753 rows  (Benign: 540,892, Attack: 68,861)

[ 9/10] 03-01-2018.csv
  → saved 331,028 rows  (Benign: 237,987, Attack: 93,041)

[10/10] 03-02-2018.csv
  → saved 1,043,640 rows  (Benign: 761,330, Attack: 282,310)

CLEAN DONE
Total Benign: 13,474,330
Total Attack: 2,364,515
Ratio: 85.1% Benign
Output dir: /kag

In [ ]:
# import shutil
# from pathlib import Path

# base = Path("/kaggle/working")

# for sub in ["kfold_splits"]:
#     p = base / sub
#     if p.exists():
#         shutil.rmtree(p)
#         print(f"Đã xóa: {p}")
#     else:
#         print(f"Không tìm thấy: {p}")

!nvidia-smi

In [7]:
# split kfold
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle, gc, warnings
warnings.filterwarnings('ignore')

CLEANED_DIR = Path('/kaggle/working/cleaned_data')
SPLIT_DIR = Path('/kaggle/working/kfold_splits')
SPLIT_DIR.mkdir(exist_ok=True)
(SPLIT_DIR / 'folds').mkdir(exist_ok=True)
(SPLIT_DIR / 'scalers').mkdir(exist_ok=True)

N_SPLITS = 5
RANDOM_STATE = 42
MAX_SAMPLES = 3_000_000   

print("="*70)
print("MEMORY-SAFE K-FOLD SPLIT (BINARY, FOR CNN/LSTM)")
print("="*70)

# 1) Scan files
csv_files = sorted(CLEANED_DIR.glob('cleaned_*.csv'))
print(f"Found {len(csv_files)} cleaned files\n")

file_info = []
total_rows = 0
for f in csv_files:
    with open(f, 'r', encoding='utf-8', errors='ignore') as fh:
        n_rows = sum(1 for _ in fh) - 1
    total_rows += n_rows
    file_info.append((f, n_rows))
    print(f"{f.name:30s} {n_rows:>10,} rows")

print(f"\nTOTAL ROWS: {total_rows:,}")

# 2) Decide sampling ratio
if total_rows > MAX_SAMPLES:
    ratio = MAX_SAMPLES / total_rows
    print(f"\n⚠ RAM limited → sampling {ratio*100:.1f}% ≈ {MAX_SAMPLES:,} rows")
else:
    ratio = 1.0
    print("\nRAM OK → use all rows")

# 3) Load with sampling
all_chunks = []
np.random.seed(RANDOM_STATE)
tot_b, tot_a = 0, 0

for f, n_rows in file_info:
    n_keep = int(n_rows * ratio)
    if n_keep < 100:
        print(f"\nSkip {f.name} (too small after sampling)")
        continue
    
    print(f"\nLoading {f.name}: {n_keep:,}/{n_rows:,} rows...", end=" ")
    if ratio < 1.0:
        # random rows to skip
        skip = sorted(np.random.choice(range(1, n_rows+1),
                                       n_rows - n_keep,
                                       replace=False))
        df = pd.read_csv(f, skiprows=skip, low_memory=False)
    else:
        df = pd.read_csv(f, low_memory=False)
    
    b = (df['Label']==0).sum()
    a = (df['Label']==1).sum()
    tot_b += b
    tot_a += a
    all_chunks.append(df)
    print(f"✓ B:{b:,}, A:{a:,}")
    del df; gc.collect()

print("\nMerging sampled chunks...")
df = pd.concat(all_chunks, ignore_index=True)
del all_chunks; gc.collect()

print("Shuffling...")
df = df.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

print(f"\nFINAL DATASET:")
print(f"  Samples: {len(df):,}")
print(f"  Benign : {tot_b:,} ({tot_b/len(df)*100:.1f}%)")
print(f"  Attack : {tot_a:,} ({tot_a/len(df)*100:.1f}%)")
print(f"  Memory : {df.memory_usage(deep=True).sum()/(1024**2):.1f} MB")

# 4) Prepare X, y
y = df['Label'].values.astype(np.int32)
df_feat = df.drop(columns=['Label'])

cat_cols = df_feat.select_dtypes(include=['object']).columns.tolist()
if cat_cols:
    print("\nEncoding categorical columns:")
    from sklearn.preprocessing import LabelEncoder
    for col in cat_cols:
        le = LabelEncoder()
        df_feat[col] = le.fit_transform(df_feat[col].astype(str))
        print(f"  {col} → encoded")

X = df_feat.values.astype(np.float32)
feat_names = df_feat.columns.tolist()

with open(SPLIT_DIR / 'scalers' / 'feature_names.pkl', 'wb') as f:
    pickle.dump(feat_names, f)

print(f"\nX shape: {X.shape}, y shape: {y.shape}")
print(f"X mem: {X.nbytes/(1024**2):.1f} MB")

# 5) K-Fold split
print("\nCreating stratified K-fold splits...")
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
fold_meta = []

for fold, (tr_idx, te_idx) in enumerate(skf.split(X, y), 1):
    print(f"\nFold {fold}/{N_SPLITS}")
    X_tr, X_te = X[tr_idx], X


MEMORY-SAFE K-FOLD SPLIT (BINARY, FOR CNN/LSTM)
Found 10 cleaned files

cleaned_02-14-2018.csv            823,447 rows
cleaned_02-15-2018.csv          1,046,810 rows
cleaned_02-16-2018.csv            911,350 rows
cleaned_02-20-2018.csv          7,948,746 rows
cleaned_02-21-2018.csv          1,031,019 rows
cleaned_02-22-2018.csv          1,046,432 rows
cleaned_02-23-2018.csv          1,046,620 rows
cleaned_02-28-2018.csv            609,753 rows
cleaned_03-01-2018.csv            331,028 rows
cleaned_03-02-2018.csv          1,043,640 rows

TOTAL ROWS: 15,838,845

⚠ RAM limited → sampling 18.9% ≈ 3,000,000 rows

Loading cleaned_02-14-2018.csv: 155,967/823,447 rows... ✓ B:126,104, A:29,863

Loading cleaned_02-15-2018.csv: 198,273/1,046,810 rows... ✓ B:188,434, A:9,839

Loading cleaned_02-16-2018.csv: 172,616/911,350 rows... ✓ B:84,191, A:88,425

Loading cleaned_02-20-2018.csv: 1,505,554/7,948,746 rows... ✓ B:1,396,790, A:108,764

Loading cleaned_02-21-2018.csv: 195,282/1,031,019 rows... ✓ B

In [8]:
#  split kfold 2
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
fold_meta = []

for fold, (tr_idx, te_idx) in enumerate(skf.split(X, y), 1):
    print(f"\nFold {fold}/{N_SPLITS}")
    X_tr, X_te = X[tr_idx], X[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]

    # Normalize
    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_tr).astype(np.float32)
    X_te = scaler.transform(X_te).astype(np.float32)

    # Save
    fold_dir = SPLIT_DIR / 'folds' / f'fold_{fold}'
    fold_dir.mkdir(exist_ok=True)

    np.save(fold_dir / 'X_train.npy', X_tr)
    np.save(fold_dir / 'y_train.npy', y_tr)
    np.save(fold_dir / 'X_test.npy',  X_te)
    np.save(fold_dir / 'y_test.npy',  y_te)

    with open(fold_dir / 'scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)

    print(f"  Saved → {fold_dir}")

print("\n✅ Split ready for CNN/LSTM training!")



Fold 1/5
  Saved → /kaggle/working/kfold_splits/folds/fold_1

Fold 2/5
  Saved → /kaggle/working/kfold_splits/folds/fold_2

Fold 3/5
  Saved → /kaggle/working/kfold_splits/folds/fold_3

Fold 4/5
  Saved → /kaggle/working/kfold_splits/folds/fold_4

Fold 5/5
  Saved → /kaggle/working/kfold_splits/folds/fold_5

✅ Split ready for CNN/LSTM training!


In [9]:
# download
import shutil
import os
from IPython.display import FileLink

# Nén toàn bộ /kaggle/working
output_filename = 'all_my_work'
shutil.make_archive(output_filename, 'zip', '/kaggle/working')

# Hiển thị thông tin
file_size = os.path.getsize(f'{output_filename}.zip') / (1024*1024)
print(f"✅ Đã nén xong!")
print(f"📦 File: {output_filename}.zip")
print(f"📊 Kích thước: {file_size:.2f} MB")
print(f"\n👇 Click link bên dưới để tải:")

# Tạo link download
FileLink(f'{output_filename}.zip')

✅ Đã nén xong!
📦 File: all_my_work.zip
📊 Kích thước: 2997.31 MB

👇 Click link bên dưới để tải:


/kaggle/working/all_my_work.zip

In [11]:
# train cnn
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path

BASE_FOLD_DIR = Path('/kaggle/working/kfold_splits/folds')
N_FOLDS = 5

def build_cnn(input_shape):
    model = keras.Sequential([
        layers.Conv1D(64, 3, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),
        layers.Dropout(0.3),

        layers.Conv1D(128, 3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),
        layers.Dropout(0.3),

        layers.Conv1D(256, 3, activation='relu'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),

        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(1, activation='sigmoid')  # Binary output
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )
    return model

fold_results = []

for fold in range(1, N_FOLDS + 1):
    print("\n" + "="*70)
    print(f"TRAINING CNN - FOLD {fold}/{N_FOLDS}")
    print("="*70)

    fold_dir = BASE_FOLD_DIR / f'fold_{fold}'
    X_train = np.load(fold_dir / 'X_train.npy')
    y_train = np.load(fold_dir / 'y_train.npy')
    X_test  = np.load(fold_dir / 'X_test.npy')
    y_test  = np.load(fold_dir / 'y_test.npy')

    # Reshape for CNN: (samples, features, 1)
    X_train_cnn = X_train.reshape(-1, X_train.shape[1], 1)
    X_test_cnn  = X_test.reshape(-1, X_test.shape[1], 1)

    print(f"X_train: {X_train_cnn.shape}, y_train: {y_train.shape}")
    print(f"X_test:  {X_test_cnn.shape}, y_test:  {y_test.shape}")

    model = build_cnn(input_shape=(X_train_cnn.shape[1], 1))
    
    checkpoint_path = f'/kaggle/working/cnn_fold{fold}_best.keras'
    callbacks = [
        keras.callbacks.ModelCheckpoint(
            checkpoint_path, monitor='val_accuracy',
            save_best_only=True, mode='max', verbose=1
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=5,
            restore_best_weights=True, verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5,
            patience=3, min_lr=1e-6, verbose=1
        )
    ]

    history = model.fit(
        X_train_cnn, y_train,
        epochs=20,
        batch_size=256,
        validation_data=(X_test_cnn, y_test),
        callbacks=callbacks,
        verbose=1
    )

    # Evaluate
    loss, acc, prec, rec = model.evaluate(X_test_cnn, y_test, verbose=0)
    f1 = 2 * prec * rec / (prec + rec + 1e-8)
    print(f"\nFOLD {fold} RESULTS:")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print(f"  F1-score:  {f1:.4f}")

    fold_results.append({
        'fold': fold,
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1
    })

# Tổng hợp kết quả
import pandas as pd

results_df = pd.DataFrame(fold_results)
print("\n" + "="*70)
print("CNN K-FOLD SUMMARY")
print("="*70)
print(results_df)
print("\nMEAN METRICS:")
print(results_df[['accuracy','precision','recall','f1']].mean())



TRAINING CNN - FOLD 1/5
X_train: (2399995, 77, 1), y_train: (2399995,)
X_test:  (599999, 77, 1), y_test:  (599999,)


I0000 00:00:1764854344.811057      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1764854344.811704      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/20


I0000 00:00:1764854351.693041     276 service.cc:148] XLA service 0x7cd39802f1e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764854351.693720     276 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1764854351.693740     276 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1764854352.241225     276 cuda_dnn.cc:529] Loaded cuDNN version 90300


  19/9375 ━━━━━━━━━━━━━━━━━━━━ 1:20 9ms/step - accuracy: 0.7062 - loss: 0.5475 - precision: 0.1839 - recall: 0.2647 

I0000 00:00:1764854357.056702     276 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


9375/9375 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9688 - loss: 0.0971 - precision: 0.9345 - recall: 0.8495
Epoch 1: val_accuracy improved from -inf to 0.98761, saving model to /kaggle/working/cnn_fold1_best.keras
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 71s 7ms/step - accuracy: 0.9688 - loss: 0.0971 - precision: 0.9345 - recall: 0.8495 - val_accuracy: 0.9876 - val_loss: 0.0501 - val_precision: 0.9857 - val_recall: 0.9306 - learning_rate: 0.0010
Epoch 2/20
9367/9375 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9871 - loss: 0.0528 - precision: 0.9913 - recall: 0.9218
Epoch 2: val_accuracy did not improve from 0.98761
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 56s 6ms/step - accuracy: 0.9871 - loss: 0.0528 - precision: 0.9913 - recall: 0.9218 - val_accuracy: 0.9875 - val_loss: 0.0491 - val_precision: 0.9933 - val_recall: 0.9224 - learning_rate: 0.0010
Epoch 3/20
9367/9375 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9874 - loss: 0.0502 - precision: 0.9925 - recall: 0.9229
Epoch 3: val_accuracy impro

In [3]:
# Kiểm tra GPU
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
print(f"Mixed precision: {tf.keras.mixed_precision.global_policy()}")

GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Mixed precision: <DTypePolicy "float32">


In [ ]:
# train_lstm.py
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import pandas as pd
import gc

# Cấu hình đường dẫn
BASE_FOLD_DIR = Path('/kaggle/input/csecicids2018-splited/folds')
N_FOLDS = 5

print("="*70)
print("LSTM TRAINING FOR CICIDS2018 - BINARY CLASSIFICATION")
print("="*70)

def build_lstm(input_shape):
    """
    Xây dựng mô hình LSTM tối ưu tốc độ cho phát hiện traffic bất thường
    """
    model = keras.Sequential([
        layers.LSTM(64, return_sequences=True, input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        
        # LSTM Layer 2 
        layers.LSTM(32, return_sequences=False),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        
        # Dense Layers 
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(name='auc')
        ]
    )
    return model

# Lưu kết quả từng fold
fold_results = []

for fold in range(1, N_FOLDS + 1):
    print("\n" + "="*70)
    print(f"TRAINING LSTM - FOLD {fold}/{N_FOLDS}")
    print("="*70)
    
    # Load dữ liệu đã split
    fold_dir = BASE_FOLD_DIR / f'fold_{fold}'
    
    print(f"\nLoading data from: {fold_dir}")
    X_train = np.load(fold_dir / 'X_train.npy')
    y_train = np.load(fold_dir / 'y_train.npy')
    X_test  = np.load(fold_dir / 'X_test.npy')
    y_test  = np.load(fold_dir / 'y_test.npy')
    
    # Reshape cho LSTM: (samples, timesteps, features)
    # LSTM cần dạng 3D: (batch_size, timesteps, features)
    # Ở đây ta coi mỗi feature như 1 timestep
    X_train_lstm = X_train.reshape(-1, X_train.shape[1], 1)
    X_test_lstm  = X_test.reshape(-1, X_test.shape[1], 1)
    
    print(f"\nData shapes:")
    print(f"  X_train: {X_train_lstm.shape}, y_train: {y_train.shape}")
    print(f"  X_test:  {X_test_lstm.shape}, y_test:  {y_test.shape}")
    
    print(f"\nClass distribution:")
    print(f"  Train - Benign: {(y_train==0).sum():,} ({(y_train==0).sum()/len(y_train)*100:.2f}%)")
    print(f"  Train - Attack: {(y_train==1).sum():,} ({(y_train==1).sum()/len(y_train)*100:.2f}%)")
    print(f"  Test  - Benign: {(y_test==0).sum():,} ({(y_test==0).sum()/len(y_test)*100:.2f}%)")
    print(f"  Test  - Attack: {(y_test==1).sum():,} ({(y_test==1).sum()/len(y_test)*100:.2f}%)")
    
    # Build model
    print("\nBuilding LSTM model...")
    model = build_lstm(input_shape=(X_train_lstm.shape[1], 1))
    
    print("\nModel Summary:")
    model.summary()
    
    # Callbacks
    checkpoint_path = f'/kaggle/working/lstm_fold{fold}_best.keras'
    callbacks = [
        keras.callbacks.ModelCheckpoint(
            checkpoint_path, 
            monitor='val_auc',
            save_best_only=True, 
            mode='max', 
            verbose=1
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=5,
            restore_best_weights=True, 
            verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.5,
            patience=3, 
            min_lr=1e-6, 
            verbose=1
        )
    ]
    
    # Training
    print(f"\nStarting training for Fold {fold}...")
    history = model.fit(
        X_train_lstm, y_train,
        epochs=20,  
        batch_size=512,  
        validation_data=(X_test_lstm, y_test),
        callbacks=callbacks,
        verbose=1
    )
    
    # Evaluate
    print(f"\nEvaluating Fold {fold}...")
    test_results = model.evaluate(X_test_lstm, y_test, verbose=0)
    loss = test_results[0]
    acc = test_results[1]
    prec = test_results[2]
    rec = test_results[3]
    auc = test_results[4]
    
    # Calculate F1-score
    f1 = 2 * prec * rec / (prec + rec + 1e-8)
    
    print(f"\n{'='*70}")
    print(f"FOLD {fold} RESULTS:")
    print(f"{'='*70}")
    print(f"  Loss:      {loss:.4f}")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print(f"  F1-score:  {f1:.4f}")
    print(f"  AUC:       {auc:.4f}")
    print(f"{'='*70}")
    
    fold_results.append({
        'fold': fold,
        'loss': loss,
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1,
        'auc': auc
    })
    
    # Clean up memory
    del X_train, y_train, X_test, y_test
    del X_train_lstm, X_test_lstm
    del model
    gc.collect()
    tf.keras.backend.clear_session()

# Tổng hợp kết quả tất cả các fold
print("\n" + "="*70)
print("LSTM K-FOLD CROSS-VALIDATION SUMMARY")
print("="*70)

results_df = pd.DataFrame(fold_results)
print("\nResults per fold:")
print(results_df.to_string(index=False))

print("\n" + "="*70)
print("MEAN METRICS ACROSS ALL FOLDS:")
print("="*70)
mean_metrics = results_df[['loss', 'accuracy', 'precision', 'recall', 'f1', 'auc']].mean()
std_metrics = results_df[['loss', 'accuracy', 'precision', 'recall', 'f1', 'auc']].std()

for metric in ['loss', 'accuracy', 'precision', 'recall', 'f1', 'auc']:
    print(f"  {metric.capitalize():12s}: {mean_metrics[metric]:.4f} ± {std_metrics[metric]:.4f}")

print("="*70)
print("✅ LSTM TRAINING COMPLETED!")
print("="*70)

# Save results to CSV
results_df.to_csv('/kaggle/working/lstm_kfold_results.csv', index=False)
print("\n📊 Results saved to: /kaggle/working/lstm_kfold_results.csv")

LSTM TRAINING FOR CICIDS2018 - BINARY CLASSIFICATION

TRAINING LSTM - FOLD 1/5

Loading data from: /kaggle/input/csecicids2018-splited/folds/fold_1

Data shapes:
  X_train: (2399995, 77, 1), y_train: (2399995,)
  X_test:  (599999, 77, 1), y_test:  (599999,)

Class distribution:
  Train - Benign: 2,041,321 (85.06%)
  Train - Attack: 358,674 (14.94%)
  Test  - Benign: 510,331 (85.06%)
  Test  - Attack: 89,668 (14.94%)

Building LSTM model...

Model Summary:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 77, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 77, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 77, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,873 (124.50 KB)

 Trainable params: 31,681 (123.75 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 77, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 77, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 77, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,873 (124.50 KB)

 Trainable params: 31,681 (123.75 KB)

 Non-trainable params: 192 (768.00 B)


Starting training for Fold 3...
Epoch 1/20
4685/4688 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9443 - auc: 0.9415 - loss: 0.1619 - precision: 0.8795 - recall: 0.7148
Epoch 1: val_auc improved from -inf to 0.97554, saving model to /kaggle/working/lstm_fold3_best.keras
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 91s 19ms/step - accuracy: 0.9443 - auc: 0.9415 - loss: 0.1618 - precision: 0.8796 - recall: 0.7149 - val_accuracy: 0.9809 - val_auc: 0.9755 - val_loss: 0.0716 - val_precision: 0.9520 - val_recall: 0.9186 - learning_rate: 0.0010
Epoch 2/20
1300/4688 ━━━━━━━━━━━━━━━━━━━━ 57s 17ms/step - accuracy: 0.9795 - auc: 0.9726 - loss: 0.0820 - precision: 0.9672 - recall: 0.8936

In [ ]:
import numpy as np
from tensorflow import keras
import pandas as pd

N_FOLDS = 5
results = []

for fold in range(1, N_FOLDS + 1):
    print(f"\n{'='*50}")
    print(f"Testing Fold {fold}")
    print('='*50)
    
    # Load model
    model = keras.models.load_model(f'/kaggle/working/lstm_fold{fold}_best.keras')
    
    # Load test data
    X_test = np.load(f'/kaggle/input/csecicids2018-splited/folds/fold_{fold}/X_test.npy')
    y_test = np.load(f'/kaggle/input/csecicids2018-splited/folds/fold_{fold}/y_test.npy')
    
    # Reshape
    X_test_lstm = X_test.reshape(-1, X_test.shape[1], 1)
    
    # Evaluate
    test_results = model.evaluate(X_test_lstm, y_test, verbose=0)
    
    results.append({
        'fold': fold,
        'accuracy': test_results[1],
        'precision': test_results[2],
        'recall': test_results[3],
        'auc': test_results[4]
    })
    
    print(f"Accuracy:  {test_results[1]:.4f}")
    print(f"Precision: {test_results[2]:.4f}")
    print(f"Recall:    {test_results[3]:.4f}")
    print(f"AUC:       {test_results[4]:.4f}")

# Tổng hợp
df = pd.DataFrame(results)
print("\n" + "="*50)
print("FINAL RESULTS (MEAN ± STD):")
print("="*50)
for col in ['accuracy', 'precision', 'recall', 'auc']:
    print(f"{col.capitalize():10s}: {df[col].mean():.4f} ± {df[col].std():.4f}")

In [3]:
# test_lstm_models.py
import numpy as np
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Cấu hình
BASE_FOLD_DIR = Path('/kaggle/input/csecicids2018-splited/folds')
MODEL_DIR = Path('/kaggle/input/model-lstm')
N_FOLDS = 5

print("="*70)
print("TESTING LSTM MODELS ON 5-FOLD CROSS-VALIDATION")
print("="*70)

# Lưu kết quả
test_results = []

for fold in range(1, N_FOLDS + 1):
    print(f"\n{'='*70}")
    print(f"TESTING LSTM - FOLD {fold}/{N_FOLDS}")
    print(f"{'='*70}")
    
    # Load model
    model_path = MODEL_DIR / f'lstm_fold{fold}_best.keras'
    print(f"Loading: {model_path.name}")
    model = keras.models.load_model(model_path)
    
    # Load test data
    X_test = np.load(BASE_FOLD_DIR / f'fold_{fold}' / 'X_test.npy')
    y_test = np.load(BASE_FOLD_DIR / f'fold_{fold}' / 'y_test.npy')
    
    # Reshape cho LSTM
    X_test_lstm = X_test.reshape(-1, X_test.shape[1], 1)
    
    print(f"Test samples: {len(X_test_lstm):,}")
    print(f"  Benign: {(y_test==0).sum():,} | Attack: {(y_test==1).sum():,}")
    
    # Evaluate
    test_loss, test_acc, test_prec, test_rec, test_auc = model.evaluate(
        X_test_lstm, y_test, 
        batch_size=512,
        verbose=0
    )
    
    # Calculate F1-score
    test_f1 = 2 * test_prec * test_rec / (test_prec + test_rec + 1e-8)
    
    # Predictions
    y_pred = (model.predict(X_test_lstm, batch_size=512, verbose=0) > 0.5).astype(int).flatten()
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Additional metrics
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # Print results
    print(f"\nMetrics:")
    print(f"  Accuracy:    {test_acc:.4f}")
    print(f"  Precision:   {test_prec:.4f}")
    print(f"  Recall:      {test_rec:.4f}")
    print(f"  F1-score:    {test_f1:.4f}")
    print(f"  AUC:         {test_auc:.4f}")
    print(f"  Specificity: {specificity:.4f}")
    
    print(f"\nConfusion Matrix:")
    print(f"  TN: {tn:>7,} | FP: {fp:>7,}")
    print(f"  FN: {fn:>7,} | TP: {tp:>7,}")
    
    # Store results
    test_results.append({
        'fold': fold,
        'loss': test_loss,
        'accuracy': test_acc,
        'precision': test_prec,
        'recall': test_rec,
        'f1': test_f1,
        'auc': test_auc,
        'specificity': specificity,
        'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp
    })
    
    # Save confusion matrix
    plt.figure(figsize=(7, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Benign', 'Attack'],
                yticklabels=['Benign', 'Attack'],
                cbar_kws={'label': 'Count'})
    plt.title(f'LSTM - Fold {fold} Confusion Matrix', fontsize=13, fontweight='bold')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(f'/kaggle/working/lstm_fold{fold}_cm.png', dpi=120)
    plt.close()
    
    # Clean up
    del model, X_test, y_test, X_test_lstm, y_pred
    tf.keras.backend.clear_session()

# ============================================================================
# TỔNG HỢP KẾT QUẢ
# ============================================================================

print("\n" + "="*70)
print("LSTM 5-FOLD CROSS-VALIDATION SUMMARY")
print("="*70)

df = pd.DataFrame(test_results)

# Hiển thị bảng kết quả
print("\nResults per fold:")
print(df[['fold', 'accuracy', 'precision', 'recall', 'f1', 'auc']].to_string(index=False))

# Mean ± STD
print("\n" + "="*70)
print("MEAN METRICS (MEAN ± STD):")
print("="*70)

metrics = ['loss', 'accuracy', 'precision', 'recall', 'f1', 'auc', 'specificity']
for metric in metrics:
    mean_val = df[metric].mean()
    std_val = df[metric].std()
    print(f"  {metric.upper():12s}: {mean_val:.4f} ± {std_val:.4f}")

# Tổng confusion matrix
print("\n" + "="*70)
print("AGGREGATED CONFUSION MATRIX:")
print("="*70)
total_tn = df['tn'].sum()
total_fp = df['fp'].sum()
total_fn = df['fn'].sum()
total_tp = df['tp'].sum()

print(f"  TN: {total_tn:>10,} | FP: {total_fp:>10,}")
print(f"  FN: {total_fn:>10,} | TP: {total_tp:>10,}")

total = total_tn + total_fp + total_fn + total_tp
overall_acc = (total_tn + total_tp) / total
print(f"\nOverall Accuracy: {overall_acc:.4f}")

# Lưu kết quả
df.to_csv('/kaggle/working/lstm_test_results.csv', index=False)
print("\n✓ Results saved: lstm_test_results.csv")

# Visualization: Metrics comparison
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(1, N_FOLDS + 1)
width = 0.15
metrics_plot = ['accuracy', 'precision', 'recall', 'f1', 'auc']
colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12', '#9b59b6']

for i, (metric, color) in enumerate(zip(metrics_plot, colors)):
    values = df[metric].values
    ax.bar(x + i*width, values, width, label=metric.capitalize(), color=color, alpha=0.8)

ax.set_xlabel('Fold', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('LSTM Metrics Across 5 Folds', fontsize=14, fontweight='bold')
ax.set_xticks(x + width*2)
ax.set_xticklabels([f'Fold {i}' for i in range(1, N_FOLDS+1)])
ax.legend(loc='lower right')
ax.set_ylim([0.90, 1.0])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('/kaggle/working/lstm_metrics_comparison.png', dpi=120)
plt.close()
print("✓ Metrics plot saved: lstm_metrics_comparison.png")

print("\n" + "="*70)
print("✅ LSTM TESTING COMPLETED!")
print("="*70)

TESTING LSTM MODELS ON 5-FOLD CROSS-VALIDATION

TESTING LSTM - FOLD 1/5
Loading: lstm_fold1_best.keras
Test samples: 599,999
  Benign: 510,331 | Attack: 89,668

Metrics:
  Accuracy:    0.9885
  Precision:   0.9970
  Recall:      0.9256
  F1-score:    0.9600
  AUC:         0.9919
  Specificity: 0.9995

Confusion Matrix:
  TN: 510,079 | FP:     252
  FN:   6,672 | TP:  82,996

TESTING LSTM - FOLD 2/5
Loading: lstm_fold2_best.keras
Test samples: 599,999
  Benign: 510,331 | Attack: 89,668

Metrics:
  Accuracy:    0.9883
  Precision:   0.9972
  Recall:      0.9244
  F1-score:    0.9594
  AUC:         0.9924
  Specificity: 0.9995

Confusion Matrix:
  TN: 510,100 | FP:     231
  FN:   6,776 | TP:  82,892

TESTING LSTM - FOLD 3/5
Loading: lstm_fold3_best.keras
Test samples: 599,999
  Benign: 510,330 | Attack: 89,669

Metrics:
  Accuracy:    0.9881
  Precision:   0.9972
  Recall:      0.9229
  F1-score:    0.9586
  AUC:         0.9920
  Specificity: 0.9995

Confusion Matrix:
  TN: 510,100 | FP:

In [2]:
# test_cnn_models.py
import numpy as np
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Cấu hình
BASE_FOLD_DIR = Path('/kaggle/input/csecicids2018-splited/folds')
MODEL_DIR = Path('/kaggle/input/model-cnn')
N_FOLDS = 5

print("="*70)
print("TESTING CNN MODELS ON 5-FOLD CROSS-VALIDATION")
print("="*70)

# Lưu kết quả
test_results = []

for fold in range(1, N_FOLDS + 1):
    print(f"\n{'='*70}")
    print(f"TESTING CNN - FOLD {fold}/{N_FOLDS}")
    print(f"{'='*70}")
    
    # Load model
    model_path = MODEL_DIR / f'cnn_fold{fold}_best.keras'
    print(f"Loading: {model_path.name}")
    model = keras.models.load_model(model_path)
    
    # Load test data
    X_test = np.load(BASE_FOLD_DIR / f'fold_{fold}' / 'X_test.npy')
    y_test = np.load(BASE_FOLD_DIR / f'fold_{fold}' / 'y_test.npy')
    
    # Reshape cho CNN
    X_test_cnn = X_test.reshape(-1, X_test.shape[1], 1)
    
    print(f"Test samples: {len(X_test_cnn):,}")
    print(f"  Benign: {(y_test==0).sum():,} | Attack: {(y_test==1).sum():,}")
    
    # Evaluate
    test_loss, test_acc, test_prec, test_rec = model.evaluate(
        X_test_cnn, y_test,
        batch_size=512,
        verbose=0
    )
    
    # Calculate F1-score
    test_f1 = 2 * test_prec * test_rec / (test_prec + test_rec + 1e-8)
    
    # Predictions
    y_pred = (model.predict(X_test_cnn, batch_size=512, verbose=0) > 0.5).astype(int).flatten()
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Additional metrics
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # Print results
    print(f"\nMetrics:")
    print(f"  Accuracy:    {test_acc:.4f}")
    print(f"  Precision:   {test_prec:.4f}")
    print(f"  Recall:      {test_rec:.4f}")
    print(f"  F1-score:    {test_f1:.4f}")
    print(f"  Specificity: {specificity:.4f}")
    
    print(f"\nConfusion Matrix:")
    print(f"  TN: {tn:>7,} | FP: {fp:>7,}")
    print(f"  FN: {fn:>7,} | TP: {tp:>7,}")
    
    # Store results
    test_results.append({
        'fold': fold,
        'loss': test_loss,
        'accuracy': test_acc,
        'precision': test_prec,
        'recall': test_rec,
        'f1': test_f1,
        'specificity': specificity,
        'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp
    })
    
    # Save confusion matrix
    plt.figure(figsize=(7, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
                xticklabels=['Benign', 'Attack'],
                yticklabels=['Benign', 'Attack'],
                cbar_kws={'label': 'Count'})
    plt.title(f'CNN - Fold {fold} Confusion Matrix', fontsize=13, fontweight='bold')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(f'/kaggle/working/cnn_fold{fold}_cm.png', dpi=120)
    plt.close()
    
    # Clean up
    del model, X_test, y_test, X_test_cnn, y_pred
    tf.keras.backend.clear_session()

# ============================================================================
# TỔNG HỢP KẾT QUẢ
# ============================================================================

print("\n" + "="*70)
print("CNN 5-FOLD CROSS-VALIDATION SUMMARY")
print("="*70)

df = pd.DataFrame(test_results)

# Hiển thị bảng kết quả
print("\nResults per fold:")
print(df[['fold', 'accuracy', 'precision', 'recall', 'f1']].to_string(index=False))

# Mean ± STD
print("\n" + "="*70)
print("MEAN METRICS (MEAN ± STD):")
print("="*70)

metrics = ['loss', 'accuracy', 'precision', 'recall', 'f1', 'specificity']
for metric in metrics:
    mean_val = df[metric].mean()
    std_val = df[metric].std()
    print(f"  {metric.upper():12s}: {mean_val:.4f} ± {std_val:.4f}")

# Tổng confusion matrix
print("\n" + "="*70)
print("AGGREGATED CONFUSION MATRIX:")
print("="*70)
total_tn = df['tn'].sum()
total_fp = df['fp'].sum()
total_fn = df['fn'].sum()
total_tp = df['tp'].sum()

print(f"  TN: {total_tn:>10,} | FP: {total_fp:>10,}")
print(f"  FN: {total_fn:>10,} | TP: {total_tp:>10,}")

total = total_tn + total_fp + total_fn + total_tp
overall_acc = (total_tn + total_tp) / total
print(f"\nOverall Accuracy: {overall_acc:.4f}")

# Lưu kết quả
df.to_csv('/kaggle/working/cnn_test_results.csv', index=False)
print("\n✓ Results saved: cnn_test_results.csv")

# Visualization: Metrics comparison
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(1, N_FOLDS + 1)
width = 0.18
metrics_plot = ['accuracy', 'precision', 'recall', 'f1']
colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']

for i, (metric, color) in enumerate(zip(metrics_plot, colors)):
    values = df[metric].values
    ax.bar(x + i*width, values, width, label=metric.capitalize(), color=color, alpha=0.8)

ax.set_xlabel('Fold', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('CNN Metrics Across 5 Folds', fontsize=14, fontweight='bold')
ax.set_xticks(x + width*1.5)
ax.set_xticklabels([f'Fold {i}' for i in range(1, N_FOLDS+1)])
ax.legend(loc='lower right')
ax.set_ylim([0.90, 1.0])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('/kaggle/working/cnn_metrics_comparison.png', dpi=120)
plt.close()
print("✓ Metrics plot saved: cnn_metrics_comparison.png")

print("\n" + "="*70)
print("✅ CNN TESTING COMPLETED!")
print("="*70)

TESTING CNN MODELS ON 5-FOLD CROSS-VALIDATION

TESTING CNN - FOLD 1/5
Loading: cnn_fold1_best.keras
Test samples: 599,999
  Benign: 510,331 | Attack: 89,668

Metrics:
  Accuracy:    0.9885
  Precision:   0.9952
  Recall:      0.9277
  F1-score:    0.9603
  Specificity: 0.9992

Confusion Matrix:
  TN: 509,933 | FP:     398
  FN:   6,484 | TP:  83,184

TESTING CNN - FOLD 2/5
Loading: cnn_fold2_best.keras
Test samples: 599,999
  Benign: 510,331 | Attack: 89,668

Metrics:
  Accuracy:    0.9884
  Precision:   0.9947
  Recall:      0.9273
  F1-score:    0.9598
  Specificity: 0.9991

Confusion Matrix:
  TN: 509,886 | FP:     445
  FN:   6,521 | TP:  83,147

TESTING CNN - FOLD 3/5
Loading: cnn_fold3_best.keras
Test samples: 599,999
  Benign: 510,330 | Attack: 89,669

Metrics:
  Accuracy:    0.9881
  Precision:   0.9956
  Recall:      0.9247
  F1-score:    0.9588
  Specificity: 0.9993

Confusion Matrix:
  TN: 509,963 | FP:     367
  FN:   6,753 | TP:  82,916

TESTING CNN - FOLD 4/5
Loading: cnn